In [1]:
import redis

In [24]:
CLIENTS_NUM = 1

In [29]:
default_responses_ack = {}
responses_ack = {}
client_responses = {}
for i in range(0, CLIENTS_NUM):
    default_responses_ack[f"client_{i}"] = False
    responses_ack[f"client_{i}"] = False
    client_responses[f"client_{i}"] = ''
print(responses_ack)

{'client_0': False}


In [2]:
subscriber = redis.Redis(
  host='redis-10323.c274.us-east-1-3.ec2.redns.redis-cloud.com',
  port=10323,
  password='hN34pvKTT8nrhOe3S9annZ6R6yNzY5Ia')

In [3]:
# Subscribe to the channel
pubsub = subscriber.pubsub()
pubsub.subscribe('messages')

In [28]:
def all_ack():
    state = True
    for sender in responses_ack:
        if(not responses_ack[sender]):
            return False
    return True

def reset_acks():
    responses_ack = default_responses_ack

In [45]:
def handle_aggregation():
    classification_result = client_responses['client_0']
    subscriber.publish('messages', f'user_response#server#{classification_result}')
    return classification_result

In [37]:
# Here, we have to send a text to the model to classify it
subscriber.publish('messages', 'input#server#Text input example')

2

In [47]:
# Listen for messages (this will block and wait for messages)
for message in pubsub.listen():
    message_type = message['type']
    if message['type'] == 'message':
        message = message['data'].decode()

        message_type = message.split("#")[0]
        sender = message.split("#")[1]
        message_content = message.split("#")[2]

        print(f'{message_type}@{sender}:', message_content)

        if(message_type == "input"):
            subscriber.publish('messages', f'client_task#server#{message_content}')

        if(message_type == "classification"):
            if sender in responses_ack.keys():
                responses_ack[sender] = True
                client_responses[sender] = message_content
            
            if all_ack():
                print('All Received -> Start aggregation')
                reset_acks()
                handle_aggregation()
                
                


input@user: Hi. how are you ?
client_task@server: Hi. how are you ?
classification@client_0: 0
All Received -> Start aggregation
user_response@server: 0
input@user: how are you
client_task@server: how are you
classification@client_0: 0
All Received -> Start aggregation
user_response@server: 0
input@user: FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
client_task@server: FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
classification@client_0: 0
All Received -> Start aggregation
user_response@server: 0
input@user: Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030
client_task@server: Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Fre

KeyboardInterrupt: 